# CityBikes

Send a request to CityBikes for the city of your choice. 

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

Put your parsed results into a DataFrame.

In [1]:
import requests
import pandas as pd
import datetime

In [ ]:
# send a request to CityBikes and get the json file
url = "http://api.citybik.es/v2/networks"
city_bikes = requests.get(url).json()
#print(city_bikes)

In [3]:
city_bikes['networks']
len(city_bikes['networks'])

611

In [57]:
 # get 10 cities
limit = 10 
for i, network in enumerate(city_bikes['networks']):
    if i >= limit:
        break
    print(network['location']['city'])

Moscow
Bærum
Siena
Maroussi
Slough
Nafplio
Parco dei Colli di Bergamo
Aigialeia
Marathon
Mantova


In [70]:
 print(network['location']['city'])

Νea Smyrni


In [8]:
# define a function to get city data

import requests

def get_city_data(city):
    url = "http://api.citybik.es/v2/networks"
    city_bikes = requests.get(url).json()
    
    list_of_city = []    
    for city_bike_dict in city_bikes['networks']:
        new_city = city_bike_dict['location']['city']
        if new_city.lower() == city.lower():
            list_of_city.append(city_bike_dict)
    return list_of_city


In [9]:

get_city_data("Hamilton, ON")

[{'company': ['Social Bicycles Inc.'],
  'gbfs_href': 'https://hamilton.socialbicycles.com/opendata/gbfs.json',
  'href': '/v2/networks/sobi-hamilton',
  'id': 'sobi-hamilton',
  'location': {'city': 'Hamilton, ON',
   'country': 'CA',
   'latitude': 43.25643601915583,
   'longitude': -79.86929655075073},
  'name': 'SoBi'}]

In [61]:
# define a function to get station data from city

def get_stations_info(city):
    station_dict = get_city_data(city)
    if not station_dict:
        print("Error:No bike company found for {}".format(city))
        return None
    
    network_address = station_dict[0]['href']
    url = "http://api.citybik.es/{}".format(network_address)
    return requests.get(url).json()['network']['stations']

In [ ]:
get_stations_info("Hamilton, ON")

In [56]:
# parse the JSON object the into a pandas dataframe


import pandas as pd

dfs = pd.DataFrame()

station_info = get_stations_info("Hamilton, ON")
station_list = []

for info in station_info:
    station_name = info['name']
    empty_slots = info['empty_slots']
    free_bikes = info['free_bikes']
    latitude = info['latitude']
    longitude = info['longitude']
    timestamp = info['timestamp']

    row = pd.DataFrame({'station_name': [station_name],'empty_slots': [empty_slots], 'free_bikes': [free_bikes],
                         'payment': [payment], 'latitude': [latitude], 'longitude': [longitude],
                        'timestamp': [timestamp]})
    dfs = pd.concat([dfs, row], ignore_index=True)
    

In [69]:
dfs.head(10)

,station_name,empty_slots,free_bikes,payment,latitude,longitude,timestamp
1,Bayfront Park,12,19,"key, applepay, androidpay, transitcard, accoun...",43.269288,-79.871327,2023-06-05T03:11:15.807000Z
42,Sanders at Hollywood,0,16,"key, applepay, androidpay, transitcard, accoun...",43.259903,-79.926481,2023-06-05T03:11:15.971000Z
50,John at Main,0,15,"key, applepay, androidpay, transitcard, accoun...",43.254529,-79.867180,2023-06-05T03:11:15.985000Z
93,Athletic Centre Bike Racks,5,14,"key, applepay, androidpay, transitcard, accoun...",43.264715,-79.915746,2023-06-05T03:11:16.093000Z
22,McMaster Mary Keyes,0,14,"key, applepay, androidpay, transitcard, accoun...",43.262460,-79.922247,2023-06-05T03:11:15.925000Z
44,Rail Trail at Stroud,0,13,"key, applepay, androidpay, transitcard, accoun...",43.253592,-79.916615,2023-06-05T03:11:15.972000Z
113,Cootes at York and King,2,13,"key, applepay, androidpay, transitcard, accoun...",43.265581,-79.949656,2023-06-05T03:11:15.989000Z
13,Hunter at Hess,3,13,"key, applepay, androidpay, transitcard, accoun...",43.255383,-79.878896,2023-06-05T03:11:15.882000Z
34,Princess Point,0,13,"key, applepay, androidpay, transitcard, accoun...",43.272991,-79.894527,2023-06-05T03:11:15.953000Z
104,Aberdeen at Studholme,1,12,"key, applepay, androidpay, transitcard, accoun...",43.252628,-79.899693,2023-06-05T03:11:16.112000Z


In [68]:
# choose the stattion that free_bike > 0 

dfs = dfs[dfs['free_bikes']>0].sort_values(by = 'free_bikes',ascending = False)
dfs.shape


(132, 7)

In [65]:
# store the dataframe to JSON file

dfs.to_json('citybike_station.json')

In [53]:
#dfs = pd.read_json('citybike_station.json')